In [1]:
import numpy as np 
import pandas as pd # ( pd.read_csv)
from collections import defaultdict
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import matplotlib.pyplot as plt
from nltk.stem import PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix, classification_report,cohen_kappa_score
import os, gensim, copy, pickle, warnings
from gensim.utils import simple_preprocess
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from nltk.corpus import stopwords
import numpy as np
from gensim import corpora, models
import seaborn as sns
np.random.seed(400)
from imblearn.over_sampling import SMOTE
import pyLDAvis
import pyLDAvis.gensim_models
import nltk

nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /Users/daisy/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/daisy/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
features = pd.read_csv('data/features.csv')
labels = pd.read_csv('data/labels.csv')
# test = pd.read_csv('data/test.csv')

### Check basic information

### Need to match the trackID to get the label for features, assuming “classic pop and rock” is different from pop

In [3]:
all_data = features.merge(labels, on=['trackID'], how='inner')
print(len(all_data),'combined data length \n', 
      len(features), 'feature length \n', 
      len(labels), 'label length')

8128 combined data length 
 8128 feature length 
 8128 label length


# Step 1: Exploratory Data Analyse and preprocessing

### The tag and title are all strings, need to convert to list and dict

In [4]:
## pre-processing by split the tag and title to list
def combine_title_and_tags(df):
    combine_title_tag_list = []
    for i in range(len(df)):
        df['title'] = df['title'].fillna('')
        df['tags'] = df['tags'].fillna('')
        combine_title_tag_list.append(" ".join([df['title'][i],df['tags'][i]]))   
    df = df.assign(text = pd.Series(combine_title_tag_list))
    df = df.drop(columns = ['tags','title'])
    return df
all_data = combine_title_and_tags(all_data)

### Imbalanced dataset, but not extremely imbalanced

# Step2: Preprocessing and prepare for modelling

In [5]:
## Split to train and test
y = all_data.pop('genre')
X_train, X_valid, y_train, y_valid = train_test_split( all_data, y, test_size=0.4, random_state=15, stratify=y)

In [6]:
print(np.shape(X_train))
X_valid= X_valid.reset_index(drop=True)
y_valid= y_valid.reset_index(drop=True)
X_valid.to_csv('test/X_valid.csv')
y_valid.to_csv('test/y_valid.csv')
X_train.to_csv('test/X_train.csv')
y_train.to_csv('test/y_train.csv')

(4876, 156)


## Check text data

In [7]:
X_train_text2 = pd.DataFrame(X_train.reset_index(drop=True))
y_train_text2 = y_train.reset_index(drop=True)

In [8]:
stop_words = stopwords.words('english')
stop_words.extend(['let','oh','hey', 'hay', 'ya', 'ooh',  'go', 'ai', 'tell', 'still','ah'
                   'ca', 'na', 'say', 'sure', 'yeah', 'tu', 'els', 'might', 'done']) 

def strip_newline(series):
    return [review.replace('\n','') for review in series]

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
        
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]


def lemmatize_stemming_2(texts):
    all_output =[]
    for j in texts:
         all_output.append([WordNetLemmatizer().lemmatize(i, pos = 'v') for i in j])
    return all_output


def bigrams(words, bi_min=15, tri_min=10):
    bigram = gensim.models.Phrases(words, min_count = bi_min)
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    return bigram_mod

def get_corpus(df):
    df['text'] = strip_newline(df.text)
    words = list(sent_to_words(df.text))
    words = lemmatize_stemming_2(words) ##new
    words = remove_stopwords(words)
    bigram_mod = bigrams(words)
    bigram = [bigram_mod[review] for review in words]
    id2word = gensim.corpora.Dictionary(bigram)
    id2word.filter_extremes(no_below=12, no_above=0.5)
    id2word.compactify()
    corpus = [id2word.doc2bow(text) for text in bigram]
    return corpus, id2word, bigram

## create vectors
def get_text_vector(df, corpuss, lda_model):
    vecs = []
    for i in range(len(df)):
        top_topics = lda_model.get_document_topics(corpuss[i], minimum_probability=0.0)
        topic_vec = [top_topics[i][1] for i in range(8)]
        topic_vec.extend([len(df.iloc[i].text)]) # length review
        vecs.append(topic_vec)
    return vecs



In [9]:
X_train_text2.columns

Index(['trackID', 'loudness', 'tempo', 'time_signature', 'key', 'mode',
       'duration', 'vect_1', 'vect_2', 'vect_3',
       ...
       'vect_140', 'vect_141', 'vect_142', 'vect_143', 'vect_144', 'vect_145',
       'vect_146', 'vect_147', 'vect_148', 'text'],
      dtype='object', length=156)

In [10]:
train_corpus, train_id2word, bigram_train = get_corpus(X_train_text2)

with open('model/train_corpus.pkl', 'wb') as f:
    pickle.dump(train_corpus, f)
with open('model/train_id2word.pkl', 'wb') as f:
    pickle.dump(train_id2word, f)
with open('model/bigram_train.pkl', 'wb') as f:
    pickle.dump(bigram_train, f)

In [11]:
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    lda_model = gensim.models.ldamulticore.LdaMulticore(
                           corpus=train_corpus,
                           num_topics=8,
                           id2word=train_id2word,
                           chunksize=20,
                           workers=7, 
                           passes=50,
                           eval_every = 1,
                           per_word_topics=True)
    lda_model.save('model/lda_model.model')

In [12]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, train_corpus, train_id2word , mds="mmds", R=30)
vis

/Users/daisy/.pyenv/versions/3.7.4/lib/python3.7/site-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.115673  0.310972       1        1  29.157790
7      0.080442  0.270946       2        1  17.671107
6      0.132873  0.036447       3        1  15.708954
1     -0.290445  0.142273       4        1  15.429473
3      0.341040  0.178127       5        1  10.353279
0     -0.081555 -0.463178       6        1   5.888875
5     -0.340733 -0.179933       7        1   3.266639
2      0.274051 -0.295653       8        1   2.523882, topic_info=           Term         Freq        Total Category  logprob  loglift
85    love_know   948.000000   948.000000  Default  30.0000  30.0000
78          get   925.000000   925.000000  Default  29.0000  29.0000
232      que_de   309.000000   309.000000  Default  28.0000  28.0000
16         love  1031.000000  1031.000000  Default  27.0000  27.0000
11           la   183.000000   183.000000  Default  26.0000  26.0000
...         ...          ...          ...      ...      ...      ...
1533       vers    43.402630   103.532550   Topic8  -4.7850   2.8100
483         fun    47.031920   132.896838   Topic8  -4.7047   2.6406
1216    explain    36.977355    73.655824   Topic8  -4.9452   2.9903
447       simpl    31.983650    96.774648   Topic8  -5.0903   2.5722
1057        dem    30.392194    82.746265   Topic8  -5.1413   2.6778

[477 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
1730      1  0.076512       abov
1730      2  0.119550       abov
1730      3  0.765120       abov
1730      4  0.038256       abov
709       3  0.870051     across
...     ...       ...        ...
1003      1  0.882025        yes
1003      2  0.017295        yes
1003      5  0.099444        yes
2136      8  0.981748  yes_smile
2512      5  0.989723      youth

[808 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 8, 7, 2, 4, 1, 6, 3])

It seems there are some overlaps between topic 1 and 2

There are a few words seems to be stop words but not in the stop words list, added the extra stop words like ['hey', 'hay', 'ya','ooh'] to the existing stop words dictionary

In [13]:
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    lda_model2 = gensim.models.ldamulticore.LdaMulticore(
                           corpus=train_corpus,
                           num_topics=16,
                           id2word=train_id2word,
                           chunksize=50,
                           workers=7, 
                           passes=100,
                           eval_every = 1,
                           per_word_topics=False)
    lda_model2.save('model/lda_model2.model')


In [14]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model2, train_corpus, train_id2word , mds="mmds", R=30)
vis

/Users/daisy/.pyenv/versions/3.7.4/lib/python3.7/site-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6      0.206350 -0.154476       1        1  15.368511
12     0.126642 -0.302928       2        1  13.527676
3     -0.003354 -0.131949       3        1  13.480274
8      0.085358  0.133233       4        1  10.417504
2      0.245505 -0.303894       5        1   9.274355
15     0.362161 -0.072308       6        1   8.980852
5      0.264236  0.090455       7        1   6.790626
13    -0.025036 -0.340962       8        1   4.850688
10    -0.221166 -0.013795       9        1   4.685792
11     0.115711  0.501716      10        1   3.755232
1      0.392897  0.222492      11        1   2.119940
7     -0.087896  0.374566      12        1   1.762512
4     -0.403977 -0.134970      13        1   1.526068
9     -0.316958  0.350135      14        1   1.313882
0     -0.317199 -0.365321      15        1   1.136784
14    -0.423276  0.148007      16        1   1.009304, topic_info=           Term         Freq        Total Category  logprob  loglift
78          get   889.000000   889.000000  Default  30.0000  30.0000
232      que_de   309.000000   309.000000  Default  29.0000  29.0000
85    love_know   961.000000   961.000000  Default  28.0000  28.0000
16         love  1027.000000  1027.000000  Default  27.0000  27.0000
820       music   233.000000   233.000000  Default  26.0000  26.0000
...         ...          ...          ...      ...      ...      ...
2325        les    24.684739    37.913028  Topic16  -4.4328   4.1668
2339         un    26.183700    53.723124  Topic16  -4.3738   3.8772
70         ador    21.721621    30.741530  Topic16  -4.5607   4.2486
374          se    28.385777   101.357855  Topic16  -4.2931   3.3231
241          ca    26.271295   244.664082  Topic16  -4.3705   2.3645

[990 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
3015     16  0.955033  abandon
1750      8  0.958535  absolut
2257      9  0.205904   action
2257     12  0.772142   action
2495      2  0.981438   addict
...     ...       ...      ...
1729      9  0.552926       yo
1729     10  0.238764       yo
1729     14  0.201064       yo
2599     14  0.985767    yo_si
457      11  0.924749     zero

[2078 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 13, 4, 9, 3, 16, 6, 14, 11, 12, 2, 8, 5, 10, 1, 15])

### Check topic and key words example

In [15]:
lda_model.print_topics(8,num_words=15)[:12]

[(0,
  '0.025*"que_de" + 0.016*"la_en" + 0.012*"un_el" + 0.012*"que" + 0.009*"corazon" + 0.009*"mi" + 0.008*"cuando" + 0.008*"te_se" + 0.008*"se" + 0.008*"esta" + 0.007*"solo" + 0.007*"el_te" + 0.007*"tus" + 0.006*"mas" + 0.006*"se_mi"'),
 (1,
  '0.014*"get" + 0.006*"know_like" + 0.006*"money" + 0.006*"drink" + 0.005*"street" + 0.005*"buy" + 0.005*"one_get" + 0.005*"like" + 0.005*"come" + 0.005*"man" + 0.005*"stick" + 0.004*"back" + 0.004*"pay" + 0.004*"town" + 0.004*"big"'),
 (2,
  '0.016*"da" + 0.014*"die" + 0.011*"noch" + 0.011*"tri_whi" + 0.011*"style" + 0.010*"art" + 0.010*"hat" + 0.009*"mind_girl" + 0.009*"fun" + 0.008*"vers" + 0.008*"yes_smile" + 0.008*"pair" + 0.007*"talkin" + 0.007*"bust" + 0.007*"explain"'),
 (3,
  '0.012*"blood" + 0.008*"god" + 0.007*"death" + 0.006*"land" + 0.006*"fire" + 0.006*"children" + 0.005*"human" + 0.005*"war" + 0.005*"burn" + 0.005*"peac" + 0.005*"power" + 0.005*"bleed" + 0.005*"forc" + 0.005*"shall" + 0.005*"us"'),
 (4,
  '0.016*"love_know" + 0.01

In [16]:
train_vecs1 = get_text_vector(X_train_text2, train_corpus, lda_model)
train_vecs2 = get_text_vector(X_train_text2, train_corpus, lda_model2)



In [17]:
print(np.shape(train_vecs2 ))
train_vecs = np.concatenate([train_vecs1, train_vecs2], axis=1)
np.shape(train_vecs)

(4876, 9)


(4876, 18)

In [18]:
### to continue
X = np.array(train_vecs)
y = np.array(y_train_text2)

In [19]:
## simulate minority groups
def smote2(X, y):
    X1, y1 = copy.deepcopy(X), copy.deepcopy(y)  # init
    sm = SMOTE(random_state=2)
    X1, y1 = sm.fit_resample(X, y)
    return X1, y1

X2,y2 = smote2(X,y)

### prepare for test text vec

In [20]:
def get_bigram(df):
    df['text'] = strip_newline(df.text)
    words = list(sent_to_words(df.text))
    words = remove_stopwords(words)
    bigram = bigrams(words)
    bigram = [bigram[i] for i in words]
    return bigram

def prepare_text_for_prediction(X_text_df, y_text_df, id2word_dict,lda_model):
    bigram_df = get_bigram(X_text_df)
    df_corpus = [id2word_dict.doc2bow(text) for text in bigram_df]
    df_vecs = get_text_vector(X_text_df, df_corpus, lda_model)
    X_text_array = np.array(df_vecs)
    y_text_array = np.array(y_text_df)
    return X_text_array, y_text_array



In [21]:
del lda_model, train_id2word, train_corpus, bigram_train

lda_model = gensim.models.ldamulticore.LdaMulticore.load('model/lda_model.model')
train_id2word =  pickle.load(open("model/train_id2word.pkl", 'rb'))
lda_model2 = gensim.models.ldamulticore.LdaMulticore.load('model/lda_model2.model')

In [22]:
X_text_array1, y_text_array1 = prepare_text_for_prediction(X_valid, y_valid, train_id2word,lda_model)
X_text_array2, y_text_array2 = prepare_text_for_prediction(X_valid, y_valid, train_id2word,lda_model2)

In [23]:
valid_vecs = np.concatenate([X_text_array1, X_text_array2], axis=1)
X_valid_vecs = np.array(valid_vecs)
valid_y= np.array(y_valid)

In [24]:
np.shape(X_valid_vecs) , np.shape(y_valid)

((3252, 18), (3252,))

In [25]:
X_valid_text_vec2, X_test_text_vec2, y_valid2, y_test2 = train_test_split(X_valid_vecs , y_valid, test_size=0.5, random_state=123)


# Step 3: modelling 

In [26]:
def grid_search_CV(grid_para, X, y, test_X, test_y):

    model_1 = GridSearchCV(XGBClassifier(eval_metric='mlogloss'),
                               grid_para, cv=3)
    model_1.fit(X, y, eval_set = [(test_X, test_y)],
                early_stopping_rounds = 10)
    print(model_1.best_score_)
    print(model_1.best_params_)
    
    {'colsample_bytree': 0.5, 'eta': 0.15, 'gamma': 0.15, 'max_depth': 7, 'num_parallel_tree': 5, 'reg_alpha': 0.5, 'reg_lambda': 0.1, 'sampling_method': 'gradient_based'}
    {'colsample_bytree': 0.5, 'eta': 0.1, 'gamma': 0.15, 'max_depth': 7, 'num_parallel_tree': 5, 'reg_alpha': 0.6, 'reg_lambda': 0.1, 'sampling_method': 'gradient_based'}
    {'colsample_bytree': 0.5, 'eta': 0.1, 'gamma': 0.1, 'max_depth': 7, 'num_parallel_tree': 5, 'reg_alpha': 0.5, 'reg_lambda': 0.1, 'sampling_method': 'gradient_based'}
    {'colsample_bytree': 0.45, 'eta': 0.1, 'gamma': 0.1, 'max_depth': 7, 'num_parallel_tree': 5, 'reg_alpha': 0.45, 'reg_lambda': 0.15, 'sampling_method': 'gradient_based'}

    xgb_m = XGBClassifier(eval_metric = ['mlogloss'],
                          objective='multi:softprob',
                          booster = 'gbtree',
                          grow_policy='lossguide',
                          max_depth=model_1.best_params_['max_depth'],
                          reg_alpha=model_1.best_params_['reg_alpha'],
                          reg_lambda=model_1.best_params_['reg_lambda'],
                          eta=model_1.best_params_['eta'],
                          num_parallel_tree = model_1.best_params_['num_parallel_tree'],
                          gamma=model_1.best_params_['gamma'],
                          sampling_method=model_1.best_params_['sampling_method'],
                          colsample_bytree=model_1.best_params_['colsample_bytree'],
                          seed=123,
                          n_jobs = -1,
                            )
    xgb_m.fit(X, y)
    xgb_m.save_model('model/xgb_model_text.model')
    return  xgb_m

In [ ]:

max_depth = [7, 5]  #5, 
eta = [.1, .15]  
gamma = [.15, .1] 
sampling_method = ['gradient_based']  
reg_lambda = [.1, .15]
colsample_bytree = [.45, .5] 
alphax = [.55, .5, .4] 
num_parallel_tree = [5] 
grid_para_2 = dict(max_depth=max_depth, reg_lambda=reg_lambda, reg_alpha=alphax,
                 colsample_bytree=colsample_bytree, eta=eta, gamma=gamma,
                 sampling_method=sampling_method, 
                 num_parallel_tree=num_parallel_tree)
xgb_model = grid_search_CV(grid_para_2, X2, y2, X_valid_text_vec2, y_valid2) #

[0]	validation_0-mlogloss:2.02337


/Users/daisy/.pyenv/versions/3.7.4/lib/python3.7/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[1]	validation_0-mlogloss:1.97586
[2]	validation_0-mlogloss:1.93720
[3]	validation_0-mlogloss:1.90324
[4]	validation_0-mlogloss:1.87467
[5]	validation_0-mlogloss:1.85030
[6]	validation_0-mlogloss:1.82599
[7]	validation_0-mlogloss:1.80635
[8]	validation_0-mlogloss:1.78778
[9]	validation_0-mlogloss:1.77035
[10]	validation_0-mlogloss:1.75442
[11]	validation_0-mlogloss:1.74072
[12]	validation_0-mlogloss:1.72915
[13]	validation_0-mlogloss:1.71861
[14]	validation_0-mlogloss:1.70796
[15]	validation_0-mlogloss:1.69752
[16]	validation_0-mlogloss:1.68887
[17]	validation_0-mlogloss:1.68152
[18]	validation_0-mlogloss:1.67444
[19]	validation_0-mlogloss:1.66794
[20]	validation_0-mlogloss:1.66253
[21]	validation_0-mlogloss:1.65703
[22]	validation_0-mlogloss:1.65215
[23]	validation_0-mlogloss:1.64743
[24]	validation_0-mlogloss:1.64380
[25]	validation_0-mlogloss:1.64012
[26]	validation_0-mlogloss:1.63655
[27]	validation_0-mlogloss:1.63302
[28]	validation_0-mlogloss:1.63029
[29]	validation_0-mlogloss:1.

/Users/daisy/.pyenv/versions/3.7.4/lib/python3.7/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[1]	validation_0-mlogloss:1.98257
[2]	validation_0-mlogloss:1.94596
[3]	validation_0-mlogloss:1.91433
[4]	validation_0-mlogloss:1.88630
[5]	validation_0-mlogloss:1.86265
[6]	validation_0-mlogloss:1.84017
[7]	validation_0-mlogloss:1.82136
[8]	validation_0-mlogloss:1.80366
[9]	validation_0-mlogloss:1.78741
[10]	validation_0-mlogloss:1.77227
[11]	validation_0-mlogloss:1.75910
[12]	validation_0-mlogloss:1.74677
[13]	validation_0-mlogloss:1.73541
[14]	validation_0-mlogloss:1.72555
[15]	validation_0-mlogloss:1.71593
[16]	validation_0-mlogloss:1.70835
[17]	validation_0-mlogloss:1.70101
[18]	validation_0-mlogloss:1.69437
[19]	validation_0-mlogloss:1.68768
[20]	validation_0-mlogloss:1.68162
[21]	validation_0-mlogloss:1.67624
[22]	validation_0-mlogloss:1.67140
[23]	validation_0-mlogloss:1.66733
[24]	validation_0-mlogloss:1.66375
[25]	validation_0-mlogloss:1.66028
[26]	validation_0-mlogloss:1.65681
[27]	validation_0-mlogloss:1.65361
[28]	validation_0-mlogloss:1.65038
[29]	validation_0-mlogloss:1.

...

[1]	validation_0-mlogloss:1.98827
[2]	validation_0-mlogloss:1.95336
[3]	validation_0-mlogloss:1.92259
[4]	validation_0-mlogloss:1.89657
[5]	validation_0-mlogloss:1.87534
[6]	validation_0-mlogloss:1.85282
[7]	validation_0-mlogloss:1.83493
[8]	validation_0-mlogloss:1.81770
[9]	validation_0-mlogloss:1.80253
[10]	validation_0-mlogloss:1.78794
[11]	validation_0-mlogloss:1.77543
[12]	validation_0-mlogloss:1.76351
[13]	validation_0-mlogloss:1.75394
[14]	validation_0-mlogloss:1.74413
[15]	validation_0-mlogloss:1.73435
[16]	validation_0-mlogloss:1.72659
[17]	validation_0-mlogloss:1.71948
[18]	validation_0-mlogloss:1.71308
[19]	validation_0-mlogloss:1.70720
[20]	validation_0-mlogloss:1.70257
[21]	validation_0-mlogloss:1.69804
[22]	validation_0-mlogloss:1.69331
[23]	validation_0-mlogloss:1.68892
[24]	validation_0-mlogloss:1.68521
[25]	validation_0-mlogloss:1.68183
[26]	validation_0-mlogloss:1.67840
[27]	validation_0-mlogloss:1.67548
[28]	validation_0-mlogloss:1.67286
[29]	validation_0-mlogloss:1.

/Users/daisy/.pyenv/versions/3.7.4/lib/python3.7/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[1]	validation_0-mlogloss:1.98327
[2]	validation_0-mlogloss:1.94597
[3]	validation_0-mlogloss:1.91399
[4]	validation_0-mlogloss:1.88520
[5]	validation_0-mlogloss:1.86181
[6]	validation_0-mlogloss:1.83761
[7]	validation_0-mlogloss:1.81875
[8]	validation_0-mlogloss:1.80017
[9]	validation_0-mlogloss:1.78288
[10]	validation_0-mlogloss:1.76792
[11]	validation_0-mlogloss:1.75476
[12]	validation_0-mlogloss:1.74307
[13]	validation_0-mlogloss:1.73190
[14]	validation_0-mlogloss:1.72136
[15]	validation_0-mlogloss:1.71148
[16]	validation_0-mlogloss:1.70319
[17]	validation_0-mlogloss:1.69569
[18]	validation_0-mlogloss:1.68840
[19]	validation_0-mlogloss:1.68165
[20]	validation_0-mlogloss:1.67525
[21]	validation_0-mlogloss:1.66990
[22]	validation_0-mlogloss:1.66455
[23]	validation_0-mlogloss:1.65930
[24]	validation_0-mlogloss:1.65533
[25]	validation_0-mlogloss:1.65112
[26]	validation_0-mlogloss:1.64757
[27]	validation_0-mlogloss:1.64407
[28]	validation_0-mlogloss:1.64097
[29]	validation_0-mlogloss:1.

/Users/daisy/.pyenv/versions/3.7.4/lib/python3.7/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[1]	validation_0-mlogloss:1.98860
[2]	validation_0-mlogloss:1.95321
[3]	validation_0-mlogloss:1.92312
[4]	validation_0-mlogloss:1.89615
[5]	validation_0-mlogloss:1.87364
[6]	validation_0-mlogloss:1.85116
[7]	validation_0-mlogloss:1.83322
[8]	validation_0-mlogloss:1.81526
[9]	validation_0-mlogloss:1.79882
[10]	validation_0-mlogloss:1.78372
[11]	validation_0-mlogloss:1.77024
[12]	validation_0-mlogloss:1.75864
[13]	validation_0-mlogloss:1.74789
[14]	validation_0-mlogloss:1.73778
[15]	validation_0-mlogloss:1.72828
[16]	validation_0-mlogloss:1.72004
[17]	validation_0-mlogloss:1.71261
[18]	validation_0-mlogloss:1.70593
[19]	validation_0-mlogloss:1.69956
[20]	validation_0-mlogloss:1.69367
[21]	validation_0-mlogloss:1.68826
[22]	validation_0-mlogloss:1.68300
[23]	validation_0-mlogloss:1.67839
[24]	validation_0-mlogloss:1.67434
[25]	validation_0-mlogloss:1.67005
[26]	validation_0-mlogloss:1.66661
[27]	validation_0-mlogloss:1.66294
[28]	validation_0-mlogloss:1.66012
[29]	validation_0-mlogloss:1.

In [ ]:
ax= plt.subplot()
cm = confusion_matrix(xgb_model.predict( X_test_text_vec2), y_test2)
sns.heatmap(cm, annot=True, fmt='g', ax=ax, cmap='Blues'); 

# labels, title and ticks
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels'); 
ax.set_title('Confusion Matrix'); 


In [ ]:
print(classification_report(xgb_model.predict( X_test_text_vec2), y_test2))

In [ ]:
print(cohen_kappa_score(xgb_model.predict(X_test_text_vec2), y_test2))

In [ ]:
pd.DataFrame(xgb_model.predict(X_valid_vecs),columns=['pred']).to_csv('test/valid_test_prediction.csv')